Code for detecting (and drawing bounding boxes around) changes made by model generating ID badges on images

In [56]:
from skimage.metrics import structural_similarity
import numpy as np
import cv2
from copy import copy

In [57]:
MIN_THRESHOLD = 50
MIN_AREA = 150

In [58]:
def get_possible_chests(image):
    """
    Function that returns human chests' locations in a given image
    Parameters:
    image -- numpy array, cv2 greysale image
    Return:
    all_faces -- numpy array n x 4, n is the number of faces
    """
    haar_face_files = ["haarcascade_frontalface_default.xml", "haarcascade_profileface.xml"]
    all_faces = None
    min_sizes = [100, 50, 20, 10, 5]
    for min_size in min_sizes:
        for haar_face_file in haar_face_files:
            haar_cascade = cv2.CascadeClassifier(haar_face_file)
            detected_faces = haar_cascade.detectMultiScale(
            image,
            scaleFactor = 1.01,
            minNeighbors = 8,
            minSize = (min_size, min_size),
        )

            if len(detected_faces) != 0:
                if all_faces is None:
                    all_faces = detected_faces
                else:
                    all_faces = np.concatenate((all_faces, detected_faces), axis=0)
        if all_faces is not None:
            break
    chests = copy(all_faces)
    if all_faces is not None:
        for chest in chests:
            new_x = chest[0] - chest[2]
            new_y = chest[1] + chest[3]
            new_w = chest[2] * 3
            new_h = int(chest[3] * 1.5)
            chest[0] = new_x
            chest[1] = new_y
            chest[2] = new_w
            chest[3] = new_h
    return all_faces, chests

def check_if_rects_overlap(x, y, w, h, x2, y2, w2, h2):
    if not (x > x2 + w2 or y > y2 + h2 or
                x + w < x2 or y + h < y2):
        return True
    else:
        return False
    
def consolidate_bounding_boxes(bb1, bb2):
    # combine bounding boxes that overlap
    return [min(bb1[0], bb2[0]),
           min(bb1[1], bb2[1]),
           max(bb1[0]+bb1[2], bb2[0]+bb2[2]) - min(bb1[0], bb2[0]),
           max(bb1[1]+bb1[3], bb2[1]+bb2[3]) - min(bb1[1], bb2[1])]

def consolidate_all_bounding_boxes(bounding_boxes):
    # consolidates all overlaping bounding boxes in a list
    while True:
        for i in range(len(bounding_boxes)):
            for j in range(len(bounding_boxes)):
                if i != j and check_if_rects_overlap(*bounding_boxes[i], *bounding_boxes[j]):
                    new_bounding_boxes = [bounding_boxes[k] for k in range(len(bounding_boxes)) if k != i and k != j]
                    new_bounding_boxes.append(consolidate_bounding_boxes(bounding_boxes[i], bounding_boxes[j]))
                    bounding_boxes = new_bounding_boxes
                    break
                
            else:
                continue
                
            break
        else:
            break
    return bounding_boxes
        
            
    
def check_if_overlaps_with_chests(x, y, w, h, chests, faces):
    # checks if found change overlaps with chest and not overlaps with face
    def check_if_not_overlap_with_face():
        for (x_face, y_face, w_face, h_face) in faces:
            if check_if_rects_overlap(x, y, w, h, x_face, y_face, w_face, h_face):
                if y + h/2 <= y_face + h_face and y + h/2 >= y_face:
                    return False
        return True
    if chests is None:
        return True
    for (x_chest, y_chest, w_chest, h_chest) in chests:
        if check_if_rects_overlap(x, y, w, h, x_chest, y_chest, w_chest, h_chest):
            if y + h/2 <= y_chest + h_chest and y + h/2 >= y_chest:
                if check_if_not_overlap_with_face():
                    return True
    return False


def load_images(img_before_path, img_after_path):
    # for loading images
    # img_before_path -- path to the image before modification (adding badge)
    # img_after_path -- path to the image after modification (adding badge)
    before = cv2.imread(img_before_path)
    after = cv2.imread(img_after_path)

    before_gray = cv2.cvtColor(before, cv2.COLOR_BGR2GRAY)
    after_gray = cv2.cvtColor(after, cv2.COLOR_BGR2GRAY)
    return before, after, before_gray, after_gray


def get_faces_chests_similarity(before, after, before_gray, after_gray):
    # function for detecting all the faces, chests, as well as similarity (difference) between image before and after modification
    faces, chests = get_possible_chests(after_gray)
    for x,y,w,h in faces:
        cv2.rectangle(before, (x, y), (x + w, y + h), (36,255,12), 2)
    for x,y,w,h in chests:
        cv2.rectangle(before, (x, y), (x + w, y + h), (36,12,255), 2)
    (score, diff) = structural_similarity(before_gray, after_gray, full=True)
    diff = (diff * 255).astype("uint8")
    #thresh = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
    thresh = cv2.threshold(diff, MIN_THRESHOLD, 255, cv2.THRESH_BINARY_INV)[1]
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    return faces, chests, contours, diff

def draw_save_badge_boxes(before, after, diff, faces, chests, contours, filename):
    # function for drawing bounding boxes over detected changes
    bounding_boxes = []
    final_bbs = []
    for c in contours:
        x,y,w,h = cv2.boundingRect(c)
        area = w*h
        if area > MIN_AREA:
            bounding_boxes.append((x,y,w,h))
    bounding_boxes = consolidate_all_bounding_boxes(bounding_boxes)
    for x,y,w,h in bounding_boxes:
        if check_if_overlaps_with_chests(x, y, w, h, chests, faces):
            final_bbs.append((x,y,w,h))
            cv2.rectangle(after, (x, y), (x + w, y + h), (36,255,12), 2)
    
    cv2.imwrite(filename, after)
    #cv2.imshow('before', before)
    cv2.imshow('after', after)
    cv2.imshow('diff', diff)

    cv2.waitKey()
    return bounding_boxes

In [59]:
def detect_badges(img_before_path, img_after_path, save_img_name):
    # main function for detecting badges
    if not save_img_name.endswith(".png"):
        save_img_name += ".png"
    before, after, before_gray, after_gray = load_images(img_before_path, img_after_path)
    faces, chests, contours, diff = get_faces_chests_similarity(before, after, before_gray, after_gray)
    bounding_boxes = draw_save_badge_boxes(before, after, diff, faces, chests, contours, save_img_name)
    return bounding_boxes

In [60]:
for i in ["", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14"]:
    a = detect_badges(f'orig{i}.png', f'altered{i}.png', f'detected{i}.png')

## Ideas to improve the algorithm:
Better face detection model (HAAR is hard to fine-tune to all the different images)   